### Fine-Tuning LLaMA-3.2 3B Instruct

This code will fine-tune the `LLaMA-3.2-3B-Instruct` model on the CounselChat dataset

In [1]:
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import torch
import pickle
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq, TextStreamer
from datasets import Dataset, DatasetDict
from unsloth import FastLanguageModel, is_bfloat16_supported, train_on_responses_only
from trl import SFTTrainer

/data/mn27889/miniconda3/envs/mental-health/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2550754/4266328469.py:5: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel, is_bfloat16_supported, train_on_responses_only


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Some links to be used for fine-tuning
1. https://www.kdnuggets.com/fine-tuning-llama-using-unsloth
2. https://www.analyticsvidhya.com/blog/2024/12/fine-tuning-llama-3-2-3b-for-rag/
3. https://www.linkedin.com/pulse/step-guide-use-fine-tune-llama-32-dr-oualid-soula-xmnff/
4. https://medium.com/@alexandros_chariton/how-to-fine-tune-llama-3-2-instruct-on-your-own-data-a-detailed-guide-e5f522f397d7 (this seems faulty since it copies the output id's within the input_ids)
5. https://drlee.io/step-by-step-guide-fine-tuning-metas-llama-3-2-1b-model-f1262eda36c8
6. https://huggingface.co/blog/ImranzamanML/fine-tuning-1b-llama-32-a-comprehensive-article
7. https://blog.futuresmart.ai/fine-tune-llama-32-vision-language-model-on-custom-datasets
8. https://github.com/meta-llama/llama-cookbook/blob/main/getting-started/finetuning/multigpu_finetuning.md



### Reading the Counsel Chat Dataset

In [4]:
with open('processed_data/counselchat_top_votes_train.pkl', 'rb') as file:
    dataset_top_votes_train = pickle.load(file)

dataset_top_votes_train.head()

,topic,question,answerText
0,relationships,I am currently suffering from erectile dysfunc...,"Hi, First and foremost, I want to acknowledge ..."
1,family-conflict,For the past week or so me and my boyfriend ha...,Forgetting one's emotions is impossible.Since ...
2,depression,I am in high school and have been facing anxie...,"Hi Helena,I felt a bit sad when I read this. T..."
3,anxiety,I'm concerned about my boyfriend. I suffer fro...,Hello! Thank you for your question. There are ...
4,spirituality,"I'm a Christian teenage girl, and I have lost ...",Having sex with your boyfriend is and was a mi...


In [5]:
with open('processed_data/counselchat_top_votes_test.pkl', 'rb') as file:
    dataset_top_votes_test = pickle.load(file)

dataset_top_votes_test.head()

,topic,question,answerText
0,relationships,I had to go to the emergency room today to get...,It is extremely frustrating when our significa...
1,marriage,What makes a healthy marriage last? What makes...,"This is a fantastic question. In one sentence,..."
2,relationships,"I'm a female freshman in high school, and this...","First off, I think it is great that you are wi..."
3,intimacy,"My wife and I are newly married, about 2 month...","You are newly married, you Have a hectic sched..."
4,legal-regulatory,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...


Creating the HuggingFace dataset using Pandas Dataframe

In [6]:
dataset_train = Dataset.from_pandas(dataset_top_votes_train)
dataset_test = Dataset.from_pandas(dataset_top_votes_test)

In [7]:
dataset = DatasetDict()
dataset['train'] = dataset_train
dataset['test'] = dataset_test

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['topic', 'question', 'answerText'],
        num_rows: 690
    })
    test: Dataset({
        features: ['topic', 'question', 'answerText'],
        num_rows: 173
    })
})

### Fine-Tuning Code

#### Loading the model and tokenizer

In [9]:
max_seq_length = 2048 
dtype = None # None for auto-detection.
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.1.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.413 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
FastLanguageModel.for_inference(model)

text_streamer = TextStreamer(tokenizer)
inputs = tokenizer("Hello", return_tensors="pt").to("cuda:0")
print(inputs)
model.generate(**inputs, max_new_tokens=1024, streamer=text_streamer)

NameError: name 'TextStreamer' is not defined

#### Forming the chat template

In [10]:
# Define a function to apply the chat template
def format_chat_template(example):
        
    messages = [
        {"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answerText']}
    ]
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    return {"text": prompt}

In [11]:
dataset_formatted = dataset.map(format_chat_template)

Map: 100%|██████████| 173/173 [00:00<00:00, 7331.07 examples/s]


In [12]:
print(dataset_formatted['train']['text'][0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Mar 2025

You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health<|eot_id|><|start_header_id|>user<|end_header_id|>

I am currently suffering from erectile dysfunction and have tried Viagra, Cialis, etc. Nothing seemed to work. My girlfriend of 3 years is very sexually frustrated. I told her that it is okay for her to have sex with other men. Is that really okay? Is it okay for my girlfriend to have sex with other men since I can't sexually perform?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hi, First and foremost, I want to acknowledge your efforts to gain (your) ideal erectile function. If the medications are not working and you have taken them as prescribed, I would encourage you to seek the help of a sex therapist as the dysfunction may be due to a psychological and/or relational issue rather than 

#### Initializing the TRL SFTTrainer and related Arguments

In [13]:
training_args = TrainingArguments(
        output_dir="./llama32-sft-fine-tune-counselchat",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        eval_strategy="steps",
        eval_steps=0.2,
        save_steps=0.5,
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset_formatted["train"],
    eval_dataset=dataset_formatted["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 173/173 [00:01<00:00, 139.22 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
print(tokenizer.decode(trainer.train_dataset['input_ids'][0]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Mar 2025

You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health<|eot_id|><|start_header_id|>user<|end_header_id|>

I am currently suffering from erectile dysfunction and have tried Viagra, Cialis, etc. Nothing seemed to work. My girlfriend of 3 years is very sexually frustrated. I told her that it is okay for her to have sex with other men. Is that really okay? Is it okay for my girlfriend to have sex with other men since I can't sexually perform?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hi, First and foremost, I want to acknowledge your efforts to gain (your) ideal erectile function. If the medications are not working and you have taken them as prescribed, I would encourage you to seek the help of a sex therapist as the dysfunction may be due to a psychological and/or relational i

#### Only Focus on the `Response Part` for the generation

In [15]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=64): 100%|██████████| 173/173 [00:00<00:00, 180.91 examples/s]


In [16]:
# The labels are created which only contain response. Left Padding is implemented and all the padding tokens are given a score of -100 to avoid loss calculation for pad_tokens
trainer.train_dataset['labels'][0]

[-100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 13347,
 11,
 5629,
 323,
 43780,
 11,
 358,
 1390,
 311,
 25670,
 701,
 9045,
 311,
 8895,
 320,
 22479,
 8,
 10728,


#### Train the model

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 690 | Num Epochs = 3 | Total steps = 258
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 2,818,747,392/3,212,749,824 (87.74% trained)
/usr/bin/ld: skipping incompatible /usr/lib/i386-linux-gnu/libcuda.so when searching for -lcuda
/usr/bin/ld: cannot find -lcuda
collect2: error: ld returned 1 exit status


CalledProcessError: Command '['/usr/bin/gcc', '/tmp/tmpdbjlnhfb/main.c', '-O3', '-shared', '-fPIC', '-o', '/tmp/tmpdbjlnhfb/cuda_utils.cpython-310-x86_64-linux-gnu.so', '-lcuda', '-L/data/mn27889/miniconda3/envs/mental-health/lib/python3.10/site-packages/triton/backends/nvidia/lib', '-L/usr/lib/x86_64-linux-gnu', '-L/usr/lib/i386-linux-gnu', '-I/data/mn27889/miniconda3/envs/mental-health/lib/python3.10/site-packages/triton/backends/nvidia/include', '-I/tmp/tmpdbjlnhfb', '-I/data/mn27889/miniconda3/envs/mental-health/include/python3.10']' returned non-zero exit status 1.

#### Saving the model and tokenizer

In [ ]:
model.save_pretrained("./llama32-sft-fine-tune-counselchat")
tokenizer.save_pretrained("./llama32-sft-fine-tune-counselchat")

### Inference

In [ ]:
FastLanguageModel.for_inference(model)

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "I don't have the will to live anymore. I don't feel happiness in anything. "}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])